# MEMORIA PROYECTO EDA - CAMPEONES DEL GRAND SLAM

#### El proyecto que presento consiste en un analisis para comprobar una serie de hipotesis basadas en los campeones masculinos de los 4 grandes torneos de la ATP, los Grand Slams, Australian Open, US Open, Roland Garros y Wimbledon 

##### Lo primero que lleve a cabo para encontrar los datos en los que basarme para realizar el analisis fue una busqueda en internet, principalmente use la web de kaggle.com
##### Aqui obtuve el Dataset principal en el que he pasado este EDA: https://www.kaggle.com/datasets/wonduk/mens-tennis-grand-slam-winner-dataset

##### Las principales bibliotecas utilizadas han sido:

In [4]:
# Para obtener informacion necesaria para completar el Dataset principal y hacer Webscraping:
import lxml
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
# Para la limpieza y el analisis de datos:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

#### Una conseguido el Dataset principal y planteadas las hipotesis, comienzo con el trabajo de "Data understanding", haciendo una primera exploracion de datos y viendo que otras neceisdad voy a tener. 

In [ ]:
import pandas as pd
df_majors = pd.read_csv('./data/Mens_Tennis_Grand_Slam_Winner.csv')

#### Al empezar con el analisis inicial concluyo que necesito una serie de datos (2 columnas practicamente con un 80 % de nulos) que no tenia forma de conseguirlos con el material disponible, y me puse a investigar y hacer un Web scraping sobre la web de la ATP (https://www.atptour.com/es) para conseguir todo los datos relativos al ranking ATP de cada uno de los jugadores entre los años 1973 y 2919.

In [ ]:
url = 'https://www.atptour.com/es/rankings/singles?RankRange=1-5000&Region=all&DateWeek=1973-09-26'
response = requests.get(url)
response

In [ ]:
from fake_useragent import UserAgent                #ya que me niega el acceso, hay que ejecutar este código
ua = UserAgent()
headers = {'User-Agent': ua.random}
response = requests.get(url, headers=headers)
response

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# Buscar la tabla
dicc1 = {"1973":[]}
tabla_principal = soup.find("table", class_="mobile-table mega-table non-live")
# Verificar si se encontró la tabla
if tabla_principal:
    # Si se encontró la tabla, buscar todas las filas dentro de ella con la clase "lower-row"
    filas = tabla_principal.find_all('tr', class_="lower-row")
    # Hacer algo con las filas encontradas
    for fila in filas:
        nombre = fila.find('a').find('span', class_='lastName').text.strip()
        #print(nombre)
        dicc1["1973"].append(nombre)
else:
    print("No se encontró la tabla principal")

#### Este trabajo me llevo más de los esperado, no conseguia hacer un analisis en el  que cambiando el año, automaticamente me sacara los jugaodres de ese año, con la commplicacion añadida de que de un año a otro cambiaban totalmente las fechas (un años era 26-9-1987 y el siguiente 29-9-1987) y viendo que no avanzaba con una solucion que fuera mas o menos automatica tuve que hacerlo año a año, creando cada diccionario y luego juntandolo por decadas.

In [ ]:
df_atp_70 = pd.read_csv('./data/nombres_atp_70.csv')
df_atp_80 = pd.read_csv('./data/nombres_atp_80.csv')
df_atp_90 = pd.read_csv('./data/nombres_atp_90.csv')
df_atp_2000 = pd.read_csv('./data/nombres_atp_2000.csv')
df_atp_2010 = pd.read_csv('./data/nombres_atp_2010.csv')

#### Cuando tuve todos los años del ranking ATP (1973-2019) hice una concatenacion con cada una de las decatas en un dataset, dando como resultado:

In [ ]:
df_ATP_1973_to_2010 = pd.concat([df_atp_70, df_atp_80, df_atp_90, df_atp_2000, df_atp_2010], axis = 1)

#### Con los 2 Dataset comence el "Data cleaning". Este fue un proceso menos tedioso que el anterior y mas automatico. 
#### Lo primero fue cambiar los nombres del Dataset principal, para a la hora de correguir los nulos que no diera problemas.

In [ ]:
# Funcion convertir nombres 
def convertir_nombre(nombre):
    partes = nombre.split()
    inicial = partes[0][0]  
    apellido = partes[-1]  
    return f'{inicial}. {apellido}'

#### A continuacion corregi algunas incoherencias que existian como que hubieran varios torneos duplicados el premio monetario de cada torneo. En el primer caso elimine el torneo duplicado y cambie los nombres por los correcto y en el segundo sustitui todo los valores de cada torneo por los valores reales de cada premio.

In [ ]:
# Cambio de Australian Open (Dec / Jan) por Australian Open
df_majors['TOURNAMENT'] = df_majors['TOURNAMENT'].str.replace(r'Australian Open \(Dec\)|Australian Open \(Jan\)', 'Australian Open', regex = True)
df_majors['TOURNAMENT'] = df_majors['TOURNAMENT'].str.replace(r'French Open', 'Roland Garros', regex = True)

In [ ]:
# Año 1977 hay un error, 2 Australian Open, 
df_majors[df_majors['YEAR'] == 1977]

,YEAR,TOURNAMENT,WINNER,RUNNER-UP,WINNER_NATIONALITY,WINNER_ATP_RANKING,RUNNER-UP_ATP_RANKING,WINNER_LEFT_OR_RIGHT_HANDED,TOURNAMENT_SURFACE,WINNER_PRIZE
179,1977,Australian Open,V. Gerulaitis,J. Lloyd,American,NaN,NaN,right,Grass,1800000
180,1977,U.S. Open,G. Vilas,J. Connors,Argentine,NaN,NaN,left,Clay - outdoors,2600000
181,1977,Wimbledon,B. Borg,J. Connors,Swedish,NaN,NaN,right,Grass / Outdoor,2400000
182,1977,Roland Garros,G. Vilas,B. Gottfried,Argentine,NaN,NaN,left,Clay,2496000
183,1977,Australian Open,R. Tanner,G. Vilas,American,NaN,NaN,left,Grass,1800000


In [ ]:
# Nuevos valores para columna "WINNER_PRIZE" con los valores reales
premios_reales = {
    'Australian Open': 1800000,
    'Roland Garros': 2496000,  
    'U.S. Open': 2600000,
    'Wimbledon': 2400000
}

In [ ]:
# Función para actualizar los valores de 'WINNER_PRIZE' basándose en el torneo
def actualizacion_precios(row):
    if row['TOURNAMENT'] in premios_reales:
        return premios_reales[row['TOURNAMENT']]
    else:
        return row['WINNER_PRIZE']

#### Con la limpieza del primero, realice el mismo proceso con el segundo. Aqui simplemente me elimie las filas y que no necesitaba y pase la columna de horizontal a vertical, para una mejor lectura y tratado posterior de los datos.

In [ ]:
# Paso a vertical 
rankings_long = df_ATP_1973_to_2010.melt(id_vars = ['Posicion'], var_name = 'YEAR', value_name = 'Player')
rankings_long['YEAR'] = rankings_long['YEAR'].astype(int) 

,Posicion,YEAR,Player
0,1,1973,I. Nastase
1,2,1973,J. Newcombe
2,3,1973,M. Orantes
3,4,1973,S. Smith
4,5,1973,J. Connors
...,...,...,...
4695,96,2019,C. Moutet
4696,97,2019,B. Klahn
4697,98,2019,E. Gerasimov
4698,99,2019,J. Munar


#### Para terminar con la limpieza y dejar los datos preparados por el analisis, sustitui todos los nulos de las columnas del ranking ATP tanto de los finalistas como de los campeones de cada uno de los torneos.

In [ ]:
# Función para rellenar los rankings
def actualizacion_ranking(row):
    year = row['YEAR']
    if pd.isna(row['WINNER_ATP_RANKING']) and (year, row['WINNER']) in ranking_dict:
        row['WINNER_ATP_RANKING'] = ranking_dict[(year, row['WINNER'])]
    if pd.isna(row['RUNNER-UP_ATP_RANKING']) and (year, row['RUNNER-UP']) in ranking_dict:
        row['RUNNER-UP_ATP_RANKING'] = ranking_dict[(year, row['RUNNER-UP'])]
    return row

#### En este punto me encontre con un nuevo problema, en el dataset principal tenia datos desde 1950 y en la ATP, datos registrados existen a partir de 1973, por lo tengo no tenia forma de consegir solucionar esos nulos y los elimine.
#### Ademas tuve que cambiar manualmente algunos datos que dieron error al ser nombres compuestos o no aparecer en los registros. 

In [ ]:
# Elimino filas anteriores a 1973 (ya que no hay datos respecto al ranking ATP)
df_majors = df_majors[df_majors['YEAR'] >= 1973]

print("El año más antiguo en df_majors ahora es:", df_majors['YEAR'].min())

El año más antiguo en df_majors ahora es: 1973


In [ ]:
# Cambio manual de nulos (ha habido problemas de lectura por nombres raros)
df_majors.at[2, 'WINNER_ATP_RANKING'] = 3.0
df_majors.at[83, 'WINNER_ATP_RANKING'] = 10.0
df_majors.at[135, 'RUNNER-UP_ATP_RANKING'] = 13.0
df_majors.at[144, 'RUNNER-UP_ATP_RANKING'] = 9.0
df_majors.at[157, 'RUNNER-UP_ATP_RANKING'] = 51.0
df_majors.at[163, 'RUNNER-UP_ATP_RANKING'] = 108.0
df_majors.at[174, 'RUNNER-UP_ATP_RANKING'] = 16.0
df_majors.at[175, 'RUNNER-UP_ATP_RANKING'] = 114.0
df_majors.at[185, 'RUNNER-UP_ATP_RANKING'] = 10.0
df_majors.at[186, 'RUNNER-UP_ATP_RANKING'] = 10.0

#### Con los datasets limpios y los datos preparados para ser analizados me puse con el analisis de las hipotesis y los resultados obtenidos para finalmente obtener una serie de conclusiones.

In [ ]:
# Hipotesis 1 - Los ganadores son siempre Top 10

# Media ATP de ganadores de cada torneo
ganadores_torneos = df_majors.groupby('TOURNAMENT')['WINNER_ATP_RANKING'].mean().round(1)

# Agrupa por 'TOURNAMENT' y obtiene todos los rankings ATP de los ganadores
rankings_por_torneo = df_majors.groupby('TOURNAMENT')['WINNER_ATP_RANKING'].apply(list)

# Medias de ranking
medias_torneos = {
    'Australian Open': 4.7,
    'Roland Garros': 3.1,
    'U.S. Open': 2.4,
    'Wimbledon': 2.6
}

colors = {
    'Australian Open': 'darkblue',
    'Roland Garros': 'saddlebrown',
    'Wimbledon': 'green',
    'U.S. Open': 'teal'
}

#  Ordenar por valores
orden_medias = dict(sorted(medias_torneos.items(), key = lambda item: item[1], reverse = True))

plt.figure(figsize = (10, 6))

bars = plt.bar(orden_medias.keys(), orden_medias.values(), color=[colors[t] for t in orden_medias.keys()])

# Linea indica top 10
plt.axhline(y = 10, color = 'r', linestyle = '--')

plt.xlabel('Torneos')
plt.ylabel('Media ranking')
plt.title('Media ranking ATP de cada Grand slam')

# Añadir valores
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 1), ha = "center", va = 'bottom')  

ax = plt.gca() 
ax.spines['top'].set_visible(False)  
ax.spines['right'].set_visible(False)

plt.show()


In [ ]:
# Hipotesis 2 - De esos Top 10, repiten victorias en esos torneo os no hay una correlacion

# DataFrame con los ganadores y el número de veces que han ganado cada torneo
numero_victorias = df_majors.groupby(['TOURNAMENT', 'WINNER']).size().reset_index(name = 'COUNTS')

# Jugadores han ganado un torneo más de una vez
victorias_repetidas = numero_victorias[numero_victorias['COUNTS'] > 1]

# Número total de torneos repetidos ganados por cada jugador
total_victorias_repetidas = victorias_repetidas.groupby('WINNER')['COUNTS'].sum().reset_index()

# Ordenar los jugadores por el número total de torneos repetidos ganados
top_victorias_repetidas = total_victorias_repetidas.sort_values(by = 'COUNTS', ascending = False).head(10)

# Solo top 10 jugadores
df_top_victorias = df_majors[df_majors['WINNER'].isin(top_victorias_repetidas['WINNER'])]

# Figura subgráficos array 2x2
fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize = (20, 20), constrained_layout = True)

# Torneos únicos
torneos = df_majors['TOURNAMENT'].unique()[:4]  

# Gráfico 
for ax, torneos in zip(axes.flatten(), torneos):

    # Jugadores en los 10 primeros
    df_torneos = df_top_victorias[df_top_victorias['TOURNAMENT'] == torneos]

    # Victorias por cada uno y ordena de mayor a menor
    victorias_orden = df_torneos['WINNER'].value_counts().index
    
    # Countplot
    sns.countplot(x = 'WINNER', data = df_torneos, ax = ax, order = victorias_orden)
    
    # Etiquetas
    ax.set_title(f'Victorias en {torneos}')
    ax.set_xlabel('Ganador')
    ax.set_ylabel('Cantidad de Victorias')

    ax.set_xticklabels(victorias_orden, rotation=45)

    # Valores 
    for p in ax.patches:
        ax.annotate(format(p.get_height(), '.0f'), 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha = 'center', va = 'center', 
                    xytext = (0, 9), 
                    textcoords = 'offset points')
        
    # Quitar las líneas 
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

#### Para finalizar el trabajo y poder realizar la presentacion, hice una presentacion en Powerpoint con todas las hipotesis planteadas y los graficos en los que me apoyo para afirmar o no esas hipotesis. 